In [1]:
!pip install tables==3.7.0

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:

import sys
sys.path.insert(1, 'CMAPSS')
import pre_process_utils

import pandas as pd
import numpy as np
import h5py


from sklearn.preprocessing import MinMaxScaler






# Load DATA

In [3]:
# Ground Truth HI
TEST = 'Data/N-CMAPSS'
PATH_IN = TEST
SOURCE = 'N-CMAPSS_DS03'

ground_truth = pd.read_hdf(PATH_IN + "/" + SOURCE+'_cycle_test.h5', 'df')
ground_truth2 = pd.read_hdf(PATH_IN + "/" + SOURCE+'_cycle_dev.h5', 'df')

ground_truth = pd.concat([ground_truth,ground_truth2]).sort_values('unit').sort_index()


g_units = ground_truth.unit.values
g_his = ground_truth.HI.values

g_cycles = []
_, s_idx = np.unique(g_units, return_index=True)
for i in g_units[np.sort(s_idx)]:
    idx = np.ravel(g_units==i)
    g_cycles.extend(np.arange(idx.sum())+1)
g_cycles = np.array(g_cycles)

In [6]:
# DS003 Data


with h5py.File(PATH_IN + "/" + SOURCE + '.h5', 'r') as hdf:
        # Development set
        W_train = np.array(hdf.get('W_dev'))             # W
        X_s_train = np.array(hdf.get('X_s_dev'))         # X_s
        # X_v_train = np.array(hdf.get('X_v_dev'))         # X_v
        T_train = np.array(hdf.get('T_dev'))             # T
        Y_train = np.array(hdf.get('Y_dev'))             # RUL  
        A_train = np.array(hdf.get('A_dev'))             # Auxiliary

        # Test set
        W_test = np.array(hdf.get('W_test'))           # W
        X_s_test = np.array(hdf.get('X_s_test'))       # X_s
        # X_v_test = np.array(hdf.get('X_v_test'))       # X_v
        T_test = np.array(hdf.get('T_test'))           # T
        Y_test = np.array(hdf.get('Y_test'))           # RUL  
        A_test = np.array(hdf.get('A_test'))           # Auxiliary
        
        # Varnams
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))  
        X_v_var = np.array(hdf.get('X_v_var')) 
        T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))
        
        # from np.array to list dtype U4/U5
        W_var = list(np.array(W_var, dtype='U20'))
        X_s_var = list(np.array(X_s_var, dtype='U20'))  
        X_v_var = list(np.array(X_v_var, dtype='U20')) 
        T_var = list(np.array(T_var, dtype='U20'))
        A_var = list(np.array(A_var, dtype='U20'))
                             
print('')
# print("Operation time (min): " , (time.clock()-t)/60)
print('')
units_train=A_train[:,0].reshape(-1,1)
cycles_train=A_train[:,1].reshape(-1,1)
fc_train = A_train[:,2].reshape(-1,1)
hi_train = A_train[:,-1]

units_test=A_test[:,0].reshape(-1,1)
cycles_test=A_test[:,1].reshape(-1,1)
fc_test = A_test[:,2].reshape(-1,1)
hi_test = A_test[:,-1]
print("XS_train",X_s_train.shape)
print(units_train.shape)



XS_train (5571277, 14)
(5571277, 1)


In [5]:
# Optional create OOD data
fc_for_training = [1]
idx_train = np.ravel(np.isin(fc_train,fc_for_training))
idx_test = np.ravel(np.isin(fc_test,fc_for_training))
X_s_train_fc = np.concatenate([X_s_train[idx_train],X_s_test[idx_test]])
w_train_fc = np.concatenate([W_train[idx_train],W_test[idx_test]])
u_train_fc = np.concatenate([units_train[idx_train],units_test[idx_test]])
c_train_fc = np.concatenate([cycles_train[idx_train],cycles_test[idx_test]])
y_train_fc = np.concatenate([Y_train[idx_train],Y_test[idx_test]])
t_train_fc = np.concatenate([T_train[idx_train],T_test[idx_test]])
hi_train_fc = np.concatenate([hi_train[idx_train],hi_test[idx_test]])

idx_train = np.ravel(~np.isin(fc_train,fc_for_training))
idx_test = np.ravel(~np.isin(fc_test,fc_for_training))
X_s_test_fc = np.concatenate([X_s_train[idx_train],X_s_test[idx_test]])
w_test_fc = np.concatenate([W_train[idx_train],W_test[idx_test]])
u_test_fc = np.concatenate([units_train[idx_train],units_test[idx_test]])
c_test_fc = np.concatenate([cycles_train[idx_train],cycles_test[idx_test]])
y_test_fc = np.concatenate([Y_train[idx_train],Y_test[idx_test]])
t_test_fc = np.concatenate([T_train[idx_train],T_test[idx_test]])
hi_test_fc = np.concatenate([hi_train[idx_train],hi_test[idx_test]])

X_s_train = X_s_train_fc
W_train = w_train_fc
units_train = u_train_fc
cycles_train = c_train_fc
Y_train = y_train_fc
T_train = t_train_fc
hi_train = hi_train_fc

X_s_test = X_s_test_fc
W_test = w_test_fc
units_test = u_test_fc
cycles_test = c_test_fc
Y_test = y_test_fc
T_test = t_test_fc
hi_test = hi_test_fc

In [7]:
#SCALING
# scaler_X = MinMaxScaler(feature_range=(-1,1))
scaler_X = MinMaxScaler()
X_s_train = scaler_X.fit_transform(X_s_train)
X_s_test = scaler_X.transform(X_s_test)

# scaler_W = MinMaxScaler(feature_range=(-1,1))
scaler_W = MinMaxScaler()
W_train = scaler_W.fit_transform(W_train)
W_test = scaler_W.transform(W_test)


In [8]:
#optional subsample data
# AND float16 precision (25% memory reduction)
sample_freq=10

# # SAMPLE
X_s_Train,W_Train,Y_Train,T_Train,units_Train,cycles_Train,hi_Train=pre_process_utils.sample_data(
    X_s_train,W_train,Y_train,T_train,units_train,cycles_train,hi_train,sample_freq)

X_s_Test,W_Test,Y_Test,T_Test,units_Test,cycles_Test,hi_Test=pre_process_utils.sample_data(
    X_s_test,W_test,Y_test,T_test,units_test,cycles_test,hi_test,sample_freq)

In [28]:
# Full window
X_windows, C_windows, U_windows = pre_process_utils.sequence_generator_full_trajectory(X_s_Train,units_Train,cycles_Train,2030)
W_windows,_,_ = pre_process_utils.sequence_generator_full_trajectory(W_Train,units_Train,cycles_Train,max = X_windows.shape[1])
Y_windows,_,_ = pre_process_utils.sequence_generator_full_trajectory(Y_Train,units_Train,cycles_Train,max = X_windows.shape[1])
T_windows,_,_ = pre_process_utils.sequence_generator_full_trajectory(T_Train,units_Train,cycles_Train,max = X_windows.shape[1])

X_windows_test, C_windows_test, U_windows_test = pre_process_utils.sequence_generator_full_trajectory(X_s_Test,units_Test,cycles_Test,X_windows.shape[1])
W_windows_test,_,_ = pre_process_utils.sequence_generator_full_trajectory(W_Test,units_Test,cycles_Test,max = X_windows.shape[1])
Y_windows_test,_,_ = pre_process_utils.sequence_generator_full_trajectory(Y_Test,units_Test,cycles_Test,max = X_windows.shape[1])
T_windows_test,_,_ = pre_process_utils.sequence_generator_full_trajectory(T_Test,units_Test,cycles_Test,max = X_windows.shape[1])

2030
2030
2030
2030
2030
2030
2030
2030
2030
2030


In [11]:
# Custom Window Size
WINDOW_LEN = 50
stride = 1

X_windows, U_windows, C_windows=pre_process_utils.sequence_generator(X_s_Train,units_Train,cycles_Train,sequence_length=WINDOW_LEN,stride = stride)
W_windows,_,_= pre_process_utils.sequence_generator(W_Train,units_Train,cycles_Train,sequence_length=WINDOW_LEN,stride = stride)
Y_windows,_,_= pre_process_utils.sequence_generator(Y_Train,units_Train,cycles_Train,sequence_length=WINDOW_LEN,option='last',stride = stride)


X_windows_test, U_windows_test,C_windows_test=pre_process_utils.sequence_generator(X_s_Test,units_Test,cycles_Test,sequence_length=WINDOW_LEN,stride = stride)
W_windows_test,_,_ = pre_process_utils.sequence_generator(W_Test,units_Test,cycles_Test,sequence_length=WINDOW_LEN,stride = stride)
Y_windows_test,_,_ = pre_process_utils.sequence_generator(Y_Test,units_Test,cycles_Test,sequence_length=WINDOW_LEN,option='last',stride = stride)


In [12]:
# SAVE DATA

np.save(PATH_IN + '/X_windows.npy',X_windows)
np.save(PATH_IN + '/U_windows.npy',U_windows)
np.save(PATH_IN + '/C_windows.npy',C_windows)
np.save(PATH_IN + '/W_windows.npy',W_windows)
np.save(PATH_IN + '/Y_windows.npy',Y_windows)


np.save(PATH_IN + '/X_windows_test.npy',X_windows_test)
np.save(PATH_IN + '/U_windows_test.npy',U_windows_test)
np.save(PATH_IN + '/C_windows_test.npy',C_windows_test)
np.save(PATH_IN + '/W_windows_test.npy',W_windows_test)
np.save(PATH_IN + '/Y_windows_test.npy',Y_windows_test)